In [7]:
from tqdm import tqdm
import numpy as np
from boltzmann_machine import BoltzmannMachine
from utils import (get_encprob_env_states, 
                   get_encprob_weight_mask, 
                   get_free_run_scores,
                   get_clamped_run_scores,
                   parity_problem_testing)

grp_size = 4
num_hnodes = 2
env_states = get_encprob_env_states(grp_size)
weight_mask = get_encprob_weight_mask(grp_size, num_hnodes) # two vis grps should not interact with each other and hidden units should not interact with each other

In [2]:
bm_list = []

noise_multiplicity = 10
learning_cycles = 1000


for i in range(10):
    bm = BoltzmannMachine(env_states, 
                        num_hnodes,
                        weight_mask=weight_mask, 
                        noisy_clamping=True,
                        logging=False,
                        num_noisyEnvState_per_envState=noise_multiplicity,
                        run_loop_count=1
                        )
    bm.learn(learning_cycles)

    last_clamped_debug_list = bm.clamped_run_debug[-1]
    last_free_run_dist = bm.free_run_debug[-1]

    print(f'#{i+1}', end=' ')
    for clamped_run_dist in last_clamped_debug_list:
        bm.clamped_run_eval(clamped_run_dist)
    bm.free_run_eval(last_free_run_dist)

    bm_list.append(bm)

  0%|          | 0/1000 [00:00<?, ?it/s]

#1 [1. 0.], [1. 1.], [0. 1.], [0. 0.], [0, 1, 2, 3, 92] W_range: (-58.0, 44.0)


  0%|          | 0/1000 [00:00<?, ?it/s]

#2 [0. 1.], [1. 1.], [0. 0.], [1. 0.], [0, 1, 2, 3, 65, 70, 109] W_range: (-48.0, 36.0)


  0%|          | 0/1000 [00:00<?, ?it/s]

...

#8 [0. 0.], [1. 1.], [1. 0.], [0. 1.], [0, 1, 2, 3, 106, 126, 131, 134] W_range: (-56.0, 40.0)


  0%|          | 0/1000 [00:00<?, ?it/s]

#9 [0. 0.], [1. 1.], [1. 0.], [0. 1.], [0, 1, 2, 3, 81, 88, 106, 109] W_range: (-48.0, 36.0)


  0%|          | 0/1000 [00:00<?, ?it/s]

#10 [0. 0.], [1. 0.], [1. 0.], [0. 1.], [0, 1, 2, 5, 6, 114] W_range: (-94.0, 64.0)


In [3]:
for bm in bm_list:
    free_run_scores = get_free_run_scores(bm)
    clamped_run_scores = get_clamped_run_scores(bm)==grp_size

    print(free_run_scores[-100:].sum(), clamped_run_scores[-10:].sum())
    # print(free_run_scores[-50:], clamped_run_scores[-100:].sum())


91 10
91 10
0 0
92 10
61 10
88 10
0 0
78 10
96 10
0 0


In [ ]:
for i, bm in enumerate(bm_list):
    scores = get_encNetwork_scores(bm)

    print(f'#{i+1}', end=' ')
    print(scores[-5:], end='; ')

    bm.free_run_eval(bm.free_run_debug[-1])

#1 [3, 3, 3, 3, 3]; [1, 2, 3, 4, 5, 52, 105, 128, 132]
#2 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 153, 156, 159, 163]
#3 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 99, 120, 142, 150]
#4 [3, 3, 3, 3, 3]; [0, 1, 3, 4, 13, 88, 101, 118]
#5 [3, 3, 3, 3, 3]; [1, 2, 3, 5, 20, 104]
#6 [4, 4, 4, 4, 4]; [0, 1, 2, 4]
#7 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 81, 106, 111]
#8 [4, 4, 4, 4, 4]; [0, 1, 2, 4, 57, 65, 96]
#9 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 90, 116, 138, 142, 174]
#10 [4, 4, 4, 4, 4]; [0, 1, 2, 3, 83, 121, 138]


#### Result
- First run - 4/10 
- 3/10
- 3/10
- 4/10; lower noise multiplocity of 100
- 3/10;  lower noise multiplocity of 50
- 3/10;  lower noise multiplocity of 50
- 4/10; noisy_multiplicty=30, learning_cycles=50*2
- 2/10; noisy_multiplicty=30, learning_cycles=50*2
- 5/5; noisy_multiplicty=20, learning_cycles=500*2 
- 5/5; noisy_multiplicty=20, learning_cycles=500*2 
- 6/10; noisy_multiplicty=10, learning_cycles=1000
- 7/10; noisy_multiplicty=10, learning_cycles=1000
- 7/10; noisy_multiplicty=""1"", learning_cycles=2000
- 9/10; noisy_multiplicty=10, learning_cycles=1000
- 9/10; noisy_multiplicty=10, learning_cycles=1000
- 7/10; noisy_multiplicty=10, learning_cycles=1000
- 6/8; noisy_multiplicty=10, learning_cycles=1000
- 9/10; noisy_multiplicty=10, learning_cycles=1000
- 7/10; ~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- 2/3; ~~
- 8/10; ~~
- 8/10; ~~
- 7/10

with clamped_run=2
- 6/10 - 
- 8/10 - 
- 9/10
- 7/10
- 7/10


